# How diverse are archaeal histones?
## Approach 
To look at the diversity of histones in archaea we need to:
0. Collect all archaeal proteins.
1. Download hmm-profile to search against.
1. Identify which proteins are histones.
2. Create database having at least gene name, sequence, genomic GC, and type of phile (use keywords and look through each species). 
3. Vizualize these histones on 3D scatterplot (length vs genomic GC vs pI).
4. Use half_blast to identify doublet proteins.
5. Use sklearn to cluster proteins:
    1. 3 dimensions: length, genomic GC, pI.
    2. n dimensions: 3-mer(or other mer of aligned protiens).
    3. n dimensions: each amino acid of aligned protiens.
6. Evaluate each cluster:
    1. Find average or median protein from each cluster.
    2. See what are the characteristic of each group.
    3. Figure out which phyla are represented in each cluster.

#### 1. Collect all archaeal proteins using slurm script on fiji. (will need to install NCBI's eDirect) ~11hrs for 6.3 mil proteins

In [7]:
!sbatch fetch_archaeal_proteins.q

/bin/sh: esearch: command not found
/bin/sh: efetch: command not found


#### The .fa file is named with a date, so as not to overwrite it next time (ex. archaeal_proteins_20191203.fa)
#### 2. Now we need to download the eukaryotic histone hmm-profile to search our database against.